In this tutorial, we will analyze Desmoid dataset from the [WORC Database](https://github.com/MStarmans91/WORCDatabase/tree/development).

The task is to correctly identify segmented lesions as either **desmoid-type fibromatosis** or **extremity soft-tissue sarcoma**.


More details on the dataset as well as the original analysis performed by their authors can be found here:

`Starmans, M. P. A. et al. (2021). The WORC* database: MRI and CT scans, segmentations, and clinical labels for 932 patients from six radiomics studies. Submitted, preprint available from https://doi.org/10.1101/2021.08.19.21262238`

`The experiments are described in the following paper: Starmans, M. P. A. et al. (2021). Reproducible radiomics through automated machine learning validated on twelve clinical applications. Submitted, preprint available from https://arxiv.org/abs/2108.08618.`

In [ ]:
import pandas as pd
import os
from pathlib import Path
from autorad.external.download_WORC import download_WORCDatabase

# Set where we will save our data and results
project_path="/home/peng/01_data/02_radiomics/demo"
base_dir = Path(project_path)
# data_dir = base_dir / "data"
# result_dir = base_dir / "results"
data_dir = base_dir / "data"
result_dir = base_dir / "results"
data_dir.mkdir(exist_ok=True, parents=True)
result_dir.mkdir(exist_ok=True, parents=True)
os.environ["AUTORAD_RESULT_DIR"] = str(result_dir)

%load_ext autoreload
%autoreload 2

# 1. check data

In [ ]:
!ls $data_dir

In [ ]:
from autorad.utils.preprocessing import get_paths_with_separate_folder_per_case

# create a table with all the paths
paths_df = get_paths_with_separate_folder_per_case(data_dir, relative=True)
paths_df.sample(5)

In [ ]:
from autorad.data import ImageDataset
import logging

logging.getLogger().setLevel(logging.ERROR)

image_dataset = ImageDataset(
    paths_df,
    ID_colname="ID",
    root_dir=data_dir,
)

# Let's take a look at the data, plotting random 10 cases
image_dataset.plot_examples(n=10, window=None)

# 2. Tumor delineation check
## 2.1 using Dice similarity to calc mask reproducibility, should set a propper cut-off value

In [ ]:
# Dice similarity
import nibabel as nib
import numpy as np
from scipy.spatial.distance import dice

def calculate_dice_scipy(mask1_path, mask2_path):
    # Load the masks
    mask1 = nib.load(mask1_path).get_fdata()
    mask2 = nib.load(mask2_path).get_fdata()

    # Binarize the masks
    mask1 = np.where(mask1 > 0, 1, 0)
    mask2 = np.where(mask2 > 0, 1, 0)

    # Flatten the masks
    mask1 = mask1.flatten()
    mask2 = mask2.flatten()

    # Calculate Dice dissimilarity
    dice_dissimilarity = dice(mask1, mask2)

    # Calculate Dice similarity
    dice_similarity = 1. - dice_dissimilarity

    return dice_similarity

# Usage:
# dice = calculate_dice_scipy('path_to_mask1.nii.gz', 'path_to_mask2.nii.gz')
# print(dice)


# 3. feature extraction

In [ ]:
from autorad.feature_extraction import FeatureExtractor

extractor = FeatureExtractor(image_dataset, extraction_params="MR_default.yaml", n_jobs=-1)
feature_df = extractor.run()

cols_to_drop = feature_df.columns[[1, 2, 3]]  # Indexing is 0-based, so 1, 2, 3 represent the 2nd, 3rd, and 4th columns
feature_df = feature_df.drop(columns=cols_to_drop)

In [ ]:
label_df = pd.read_csv(data_dir / "labels.csv")
label_df.sample(5)

In [ ]:
# Check unique IDs in both dataframes

unique_feature_ids = feature_df['ID'].unique()
unique_label_ids = label_df['patient_ID'].unique()

# Find IDs in feature_df that are not in label_df
missing_ids = set(unique_feature_ids) - set(unique_label_ids)

print(f"Missing IDs: {missing_ids}")

In [ ]:
merged_feature_df = feature_df.merge(label_df, left_on="ID",
    right_on="patient_ID", how="left")

# merged_feature_df = feature_df.merge(label_df, left_on="ID",
#     right_on="patient_ID", how="right")
merged_feature_df.to_csv(data_dir / "features.csv", index=False)
merged_feature_df = pd.read_csv(data_dir / "features.csv") # Read back the saved file to make sure it was saved correctly


In [ ]:
# simulate some random data to demonstrate the feature icc calculation

import numpy as np

# Get the first column name
first_column = merged_feature_df.columns[0]
# Get the last two column names
last_two_columns = merged_feature_df.columns[-2:]
# Drop the first and last two columns
merged_feature_df_tmp = merged_feature_df.drop(columns=[first_column, *last_two_columns])
std_dev = merged_feature_df_tmp.std()


merged_feature_df.shape
random_df = pd.DataFrame(np.random.rand(*merged_feature_df_tmp.shape), columns=merged_feature_df_tmp.columns)* std_dev

merged_feature_df_2 = merged_feature_df_tmp + random_df

# 4. feature preprocess
## 4.1 intraclass correlation coefficients (ICCs) from different masks delineated by different doctor

In [ ]:
import pandas as pd
import pingouin as pg

# Ensure both dataframes have the same columns in the same order
assert set(merged_feature_df_tmp.columns) == set(merged_feature_df_2.columns)
merged_feature_df_tmp = merged_feature_df_tmp[merged_feature_df_2.columns]

# Initialize an empty dictionary to store the ICC values
icc_values = {}
# Loop over the columns
for column in merged_feature_df_tmp.columns:
    # Create a new dataframe that combines the data from the same column in both dataframes
    df_tmp = merged_feature_df_tmp[[column]].copy()
    df_tmp['rater'] = 'rater1'
    df_tmp['subject'] = df_tmp.index
    df_2 = merged_feature_df_2[[column]].copy()
    df_2['rater'] = 'rater2'
    df_2['subject'] = df_2.index

    df = pd.concat([df_tmp, df_2])
    # df[column] = pd.to_numeric(df[column], errors='coerce')
    # # Calculate the ICC and store the result in the dictionary
    icc = pg.intraclass_corr(data=df, targets='subject', raters='rater', ratings=column,nan_policy="omit")['ICC'].values[2] # ICC ICC(3,1)
    icc_values[column] = icc

In [ ]:
icc_df = pd.DataFrame(list(icc_values.items()), columns=['Features', 'ICC'])
icc_selected = icc_df[icc_df['ICC'] > 0.965]
print(icc_selected.index)
print(icc_selected['Features'])

In [ ]:
feature_icc_selected = merged_feature_df[icc_selected['Features']]

feature_icc_selected.to_csv(data_dir / "feature_icc_selected.csv", index=False)
# feature_icc_selected['ID'] = merged_feature_df['ID']
# feature_icc_selected['diagnosis'] = merged_feature_df['diagnosis']


## 4.2 variance of each feature value 
low variance (<75%) will be removed

In [ ]:
# calculate variance of each feature
# low variance (<75%) will be removed

from sklearn.feature_selection import VarianceThreshold

# Initialize a VarianceThreshold feature selector
selector = VarianceThreshold(threshold=0.1)  # Adjust the threshold as needed

# Fit the selector to the data and transform the data
feature_icc_var_selected = selector.fit_transform(feature_icc_selected)

# Convert back to DataFrame
feature_icc_var_selected = pd.DataFrame(feature_icc_var_selected, columns=feature_icc_selected.columns[selector.get_support()])

## 4.3 chi2 of each feature value 
best 10 feature based chi2 was selected

In [ ]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
# Find the minimum value in the DataFrame
min_value = feature_icc_var_selected.min().min()
# If the minimum value is less than 0, shift all values by its absolute value
if min_value < 0:
    feature_icc_var_selected += abs(min_value)

selector = SelectKBest(chi2, k=10)

feature_icc_var_chi2_selected = selector.fit_transform(feature_icc_var_selected, merged_feature_df['diagnosis'])
feature_icc_var_chi2_selected.shape
feature_icc_var_chi2_selected = pd.DataFrame(feature_icc_var_chi2_selected, columns=feature_icc_var_selected.columns[selector.get_support()])


In [ ]:
print(feature_icc_var_chi2_selected.columns)


## 4.4 mutual information of each feature value 
best 10 feature based  mutual information was selected

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Initialize a SelectKBest feature selector
selector = SelectKBest(mutual_info_classif, k=10)  # Adjust the number of features as needed

# Fit the selector to the data and transform the data
feature_selected = selector.fit_transform(feature_icc_var_selected, merged_feature_df['diagnosis'])

# Convert back to DataFrame
feature_icc_var_mi_selected = pd.DataFrame(feature_selected, columns=feature_icc_var_selected.columns[selector.get_support()])

# Get the names of the selected features
selected_features = feature_icc_var_selected.columns[selector.get_support()]

print('Selected features:', selected_features)

## 4.5 variance inflation factor (VIF)

If you want to select features that are least correlated with each other, you can use the concept of variance inflation factor (VIF). VIF measures the correlation and multicollinearity of features. A high VIF value for a feature means that feature is highly correlated with other features.

In [ ]:
# calculate Pearson correlation between features
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(feature_icc_var_selected.values, i) for i in range(feature_icc_var_selected.shape[1])]
vif["features"] = feature_icc_var_selected.columns

# Sort by VIF Factor in ascending order
vif = vif.sort_values("VIF Factor")

# Select the features with the lowest VIF
selected_features = vif["features"].head(10)  # Adjust the number of features as needed

print('Selected features:', selected_features)

## 4.6 Pearson correlation coefficient
# large correlation coefficient (>0.75) will be removed

## 4.4 t-test or Mann-Whitney U test

In [ ]:
# t-test between groups or Mann-Whitney U test
from scipy.stats import mannwhitneyu,ttest_ind,shapiro

# Define an empty list to hold the selected features
selected_features = []

# Perform the Mann-Whitney U test for each feature
for feature in feature_icc_var_selected.columns:
    group1 = feature_icc_var_selected.loc[merged_feature_df['diagnosis'] == 0, feature]
    group2 = feature_icc_var_selected.loc[merged_feature_df['diagnosis'] == 1, feature]
    
    # Test the normality of the feature in each group
    _, p1 = shapiro(group1)
    _, p2 = shapiro(group2)
    
    # If both groups are normally distributed, perform the t-test
    if p1 > 0.05 and p2 > 0.05:
        stat, p = ttest_ind(group1, group2)
    # Otherwise, perform the Mann-Whitney U test
    else:
        stat, p = mannwhitneyu(group1, group2)
    
    # If the p-value is less than 0.05, add the feature to the list of selected features
    if p < 0.05:
        selected_features.append(feature)

print('Selected features:', selected_features)

# 5. generate feature_dataset for model trainning

## 5.1 generate feature_dataset

In [ ]:
from autorad.data import FeatureDataset
# merged_feature_df=pd.concat([merged_feature_df.iloc[:100, :300], merged_feature_df.iloc[:100, -2:]], axis=1)
feature_dataset = FeatureDataset( 
    merged_feature_df,
    target="diagnosis",
    ID_colname="ID"
)

## 5.2 Split the data into training/validation/test sets with stratification:

In [ ]:
splits_path = result_dir / "splits.yaml"
feature_dataset.split(method="train_with_cross_validation_test", save_path=splits_path)

In [ ]:
from autorad.models import MLClassifier

models = MLClassifier.initialize_default_sklearn_models()

models = [models[1]]
print(models)

In [ ]:
from autorad.preprocessing import run_auto_preprocessing

run_auto_preprocessing(
    data=feature_dataset.data,
    use_feature_selection=True,
    # feature_selection_methods=["anova", "lasso", "boruta"], # add None for optional no feature selection
    feature_selection_methods=["anova"],
    use_oversampling=None,
    result_dir=result_dir,
    )

In [ ]:
from autorad.preprocessing import run_auto_preprocessing

run_auto_preprocessing(
    data=feature_dataset.data,
    use_feature_selection=True,
    # feature_selection_methods=["anova", "lasso", "boruta"], # add None for optional no feature selection
    feature_selection_methods=["boruta"],
    use_oversampling=False,
    result_dir=result_dir,
    )

In [ ]:
from autorad.training import Trainer

trainer = Trainer(
    dataset=feature_dataset,
    models=models,
    result_dir=result_dir,
)

experiment_name = "WORC_tutorial"

trainer.set_optimizer("optuna", n_trials=200)
trainer.run(auto_preprocess=True, experiment_name=experiment_name)

In [ ]:
from autorad.utils import mlflow_utils

mlflow_utils.start_mlflow_server()

Now you should be able to inspect the training results in MLFlow dashboard at the shown address (http://localhost:8000 unless already taken). Make sure WORC_tutorial is selected as the experiment in the dashboard. 

The dashboard should look like the one below. You can click on a specific run to show more details and its artifacts.
You can also rerun preprocessing and training with different parameters, which should create a new run in MLFlow. Finally, the best existing run will be selected for evaluation.

In [ ]:
from IPython.display import Image
Image(filename='mlflow.png') # Example

### Let's evaluate the model on the unseen test dataset:

In [ ]:
from autorad.visualization import plotly_utils
from autorad.inference import infer_utils
from autorad import evaluation

artifacts = infer_utils.get_artifacts_from_best_run(experiment_name)

result_df = evaluation.evaluate_feature_dataset(
    dataset=feature_dataset,
    model=artifacts["model"],
    preprocessor=artifacts["preprocessor"],
    split="test"
)

plotly_utils.plot_roc_curve(result_df.y_true, result_df.y_pred_proba)

Waterfall plot below can show us how many cases were classified:
- correctly as fibromatosis (negative) -> green bars facing downward
- falsely as fibromatosis -> red bars facing downward
- correctly as sarcoma -> red bars facing upward
- falsely as fibromatosis -> green bars facing upward

In [ ]:
plotly_utils.plot_waterfall(result_df.y_true, result_df.y_pred_proba)

In [ ]:
print(result_df)

Let's see if the model managed to overfit the training set:

In [ ]:
train_result_df = evaluation.evaluate_feature_dataset(
    dataset=feature_dataset,
    model=artifacts["model"],
    preprocessor=artifacts["preprocessor"],
    split="train"
)

plotly_utils.plot_roc_curve(train_result_df.y_true, train_result_df.y_pred_proba)

And, for reference, validation set used in hyperparameter optimization:

In [ ]:
val_result_df = evaluation.evaluate_feature_dataset(
    dataset=feature_dataset,
    model=artifacts["model"],
    preprocessor=artifacts["preprocessor"],
    split="val"
)

plotly_utils.plot_roc_curve(val_result_df.y_true, val_result_df.y_pred_proba)

## Inference on new data

1. Simple inference from image-mask pair:

In [ ]:
from autorad.inference import infer

img_path = data_dir / "Desmoid-046" / "image.nii.gz"
seg_path = data_dir / "Desmoid-046" / "segmentation.nii.gz"


inferrer = infer.Inferrer(
    extraction_config=artifacts["extraction_config"],
    model=artifacts["model"],
    preprocessor=artifacts["preprocessor"],
    result_dir=result_dir,
)

# Get class probability only
pred = inferrer.predict_proba(img_path, seg_path)
print(f"Probability of the case being sarcoma is: {pred:.3f}")

2. More comprehensive inference from image-mask pair:

In [ ]:
from autorad.inference import infer_utils


feature_df, X_preprocessed, pred = inferrer.predict_proba_with_features(img_path, seg_path)

print(f"Prediction: {pred:.3f}")
print("Features:")
feature_df

In [ ]:
infer_utils.plot_shap_waterfall(artifacts["explainer"], X_preprocessed)

The SHAP plot above shows which features contributed most to the predicted probability of the lesion being sarcoma